<a href="https://colab.research.google.com/github/Eylz-Liu/ALM-study/blob/main/RiskIntegrity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1 - Importation de tirage


In [4]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance
import shap
import statsmodels.api as sm

# Définir les bornes de colonnes
col_start = 1   # [ 2025(1) - 2034(10) ]  [ 2025(11) - 2044(20) ]
col_end   = 43  # correspond à la colonne 11

In [1]:
!pip install cx_Oracle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cx_Oracle: filename=cx_oracle-8.3.0-cp312-cp312-linux_x86_64.whl size=854023 sha256=d3f1859cff0cf9edcb3015df49a1aee552d7f2904e88e1dc75d2650b7a5e8fcd
  Stored in directory: /root/.cache/pip/wheels/c2/36/74/ca03900db8246dfd872cb23f51299a034ff843b7c15e87b5dd
Successfully built cx_Oracle


In [2]:
!pip install pyodbc  # SQL server

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 6.5 MB/s eta 0:00:00


In [5]:
!apt-get install -y libaio1
!wget https://download.oracle.com/otn_software/linux/instantclient/instantclient-basiclite-linux.x64-21.13.0.0.0.zip
!unzip instantclient-basiclite-linux.x64-21.13.0.0.0.zip
!mkdir -p /usr/lib/oracle
!mv instantclient_21_13 /usr/lib/oracle/21

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libaio1
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 7,176 B of archives.
After this operation, 37.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libaio1 amd64 0.3.112-13build1 [7,176 B]
Fetched 7,176 B in 0s (104 kB/s)
Selecting previously unselected package libaio1:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libaio1_0.3.112-13build1_amd64.deb ...
Unpacking libaio1:amd64 (0.3.112-13build1) ...
Setting up libaio1:amd64 (0.3.112-13build1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is 

In [3]:
import cx_Oracle

dsn = cx_Oracle.makedsn("host", 1471, service_name="service")
conn = cx_Oracle.connect(user="A070359", password="Sky_94.0", dsn=dsn)

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "libclntsh.so: cannot open shared object file: No such file or directory". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help

In [ ]:
import pandas as pd

contexts = pd.read_sql("select CONTEXT_ID, DESCRIPTION, REPORTING_DATE from V_CONTEXTS", conn)
print(contexts)

In [ ]:
results = []

for idx, row in contexts.iterrows():
    ctx_id = row["CONTEXT_ID"]
    ctx_name = row["CONTEXT_NAME"]

    sql = f"""
        SELECT '{ctx_id}' AS CONTEXT_ID,
               SOUS_MODULE,
               SCR_BRUT
        FROM V_S2C_SCR_LIFE_RESULT
        WHERE CONTEXT_ID = {ctx_id}
    """
    df = pd.read_sql(sql, conn)
    df["CONTEXT_NAME"] = ctx_name
    results.append(df)

final_df = pd.concat(results, ignore_index=True)

final_df.to_csv("SCR_LIFE_RESULT_ALL_CONTEXTS.csv", index=False)

In [6]:
!pip install oracledb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.0 MB/s eta 0:00:00


In [10]:
import oracledb

# Paramètres de connexion
username = "A070359"       # ton utilisateur
password = "Sky_94.0"      # ton mot de passe
host = "zla7282v"          # le serveur Oracle
port = 1471                # le port que tu avais mentionné
service = "SS2CP_BA_01"    # le service name

# Construction du DSN
dsn = f"{host}:{port}/{service}"

# Connexion en mode thin (pas besoin d'Oracle Client)
conn = oracledb.connect(user=username, password=password, dsn=dsn)
print("Connexion réussie à RiskIntegrity !")


gaierror: [Errno -2] Name or service not known